In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from googletrans import Translator
import spacy
import en_core_web_sm
from collections import Counter
from spacy import displacy 
import locationtagger
from geopy.geocoders import Nominatim
from functools import partial
from geopy.extra.rate_limiter import RateLimiter
import numpy
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime

In [4]:
class News():

    def __init__(self):
        
        self.Url_webdriver = r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe'

    def Set_up_WebScraping(self,url):
    
        driver = webdriver.Chrome(self.Url_webdriver)
        driver.get(url)

        for i in range(45):
            driver.execute_script("scrollBy(0,+500);")
            time.sleep(1)
        
        return driver.page_source
    
    def YahooUpdateNews(self,share):

        url = requests.get(f"https://finance.yahoo.com/quote/{share}")
        chest,tag_share_news  = [],{}
        soup = BeautifulSoup(url.text, 'html')

        for div_all in soup.find_all('div', {'id': 'quoteNewsStream-0-Stream'}):

            for li in div_all.find_all('li', {'class': 'js-stream-content Pos(r)'}):

                for l in li.find_all('div', {'class': 'Py(14px) Pos(r)'}):

                    df,st,share_tag  = {},'',{}
                    
                    tag_a = l.find('a')
                    
                    # Img
                    tag_img = l.find('img')
                    if str(tag_img) != 'None': url_img = l.find('img')['src']
                    else : url_img = 'https://www.lifewire.com/thmb/yx5oJUJ4fA1TQ0h0pl9FM7Kc4Fo=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/yahoo-logo-2019-879b7bed612d4bbc97065dce2a0f2d73.png'

                    # Get Content -----------------------------------------------------
                    
                    soup_content = BeautifulSoup((requests.get('https://finance.yahoo.com/' + tag_a['href'])).text, 'html')

                    for li_ in soup_content.find_all('p'):
                        text = li_.text
                        if text != '':st += li_.text + '\n'

                    try :        

                        df['Date'],df['Title'],df['Description'],df['Img'],df['Link'],df['Source'],df['Content']  = datetime.datetime.strptime(soup_content.find('time')['datetime'], '%Y-%m-%dT%H:%M:%S.%f%z').strftime('%d/%m/%Y'),tag_a.text,l.find('p').text,url_img,'https://finance.yahoo.com/' + tag_a['href'],'yahoo',st
                        share_news = [i.text for i in soup_content.find_all('div', {'class': 'xray-card-row-title'})]
                        tag_share_news[df['Title']] = share_news
                        print(share_news)
                        chest.append(df)
                    except TypeError:
                        pass

        return chest,tag_share_news
    
    def Kaohoon_news(self,triker):

        chest,I,tag_share_news  = [],0,{}

        url = requests.get(f"https://www.kaohoon.com/?s={triker}")
        soup = BeautifulSoup(url.text, 'html')
        for ultag in soup.find_all('ul', {'class': 'posts-items'}):

            for li in ultag.find_all('li'):

                df,st  = {},''
                text = (li.text).split('\n')

                df['Date'],df['Title'],df['Description'],df['Img'],df['Link'],df['Source'] = text[3],text[4],text[5],li.find('img')['src'],li.find('a')['href'],'kaohoon'

                soup = BeautifulSoup((requests.get(f"{df['Link']}")).text, 'html')

                div = soup.find('div', {'class': 'entry-content entry clearfix'})

                if str(div) == 'None' : return {}
                for li in div.find_all('p'):
                    text = li.text
                    if text != '':
                        st += li.text + '\n'
                
                df['Content'] = st
                tag_share_news[df['Title']] = triker

                if st != '': chest.append(df)
                
        return chest,tag_share_news
    
    def get_id_News(title):
        # connect to the database
        conn = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V2.sqlite')
        c = conn.cursor()
        id = ''
        # search for words in the table
        c.execute(f"""SELECT * FROM News WHERE title="{title}" """)
        results = c.fetchall()

        if results != []: id = str(results[0][0])
            
        else : id = 'None'

        # close the database connection
        conn.close()

        return id
        
    def add_data_to_db(self,table_name, column ,data):

        conn = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V2.sqlite')
        c = conn.cursor()

        # Create a string with placeholders for each value in the data tuple
        placeholders = ",".join(["?" for _ in data])

        # Construct the SQL query string
        query_string = f"INSERT INTO {table_name} ({column}) VALUES ({placeholders})"

        # Execute the query and commit the changes
        c.execute(query_string, data)
        conn.commit()

        # Close the connection
        conn.close()

    def save_into_db(self,table,dic):

        df = pd.DataFrame(dic)
        con = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V2.sqlite')
        cur = con.cursor()    
        newshare = df.to_sql(table,con,if_exists='append', index=False)
        con.commit()
        con.close

class find_location_share_News():

    

    def translate_text(self,text):

        detector = Translator()

        dec_lan = ''
        for sec in range(int(len(text)/1000)+1):

            dec_lan += detector.translate(text[1000*sec:1000*(sec+1)],des='en').text

        return dec_lan


    def Separate_words(self,text):

        w = text.split(r'\n')
        for content in w : text+= content + ' '

        nlp = spacy.load('en_core_web_sm')
        # nlp = en_core_web_sm.load()
        doc = nlp(text)

        word = []

        for token in doc.ents:
            
            if token.label_ != 'DATE' and token.label_ !='CARDINAL' and token.label_ !='TIME':
                # print(token.text,token.label_)     
                word += token.text.split('"')

        return list(numpy.unique(word))

    def filter_special_2(self,words):
        # Define regular expression pattern to match words
        units,sequence,share_word = ['kg','kilograms',"percent","billion","%","megawatts",'trillion','meters','#','million','cents'],['1st','2nd','3rd','first','second','third'],['Q1/','Q2/','Q3/','Q4/','SET','NASDAQ']
        word_filter = []

        # Find all matches of the pattern in the text
        for word in words :
            
            if word not in sequence :

                s = 0
                
                if "%" in word or '$' in word : s = 1

                Subwords = word.split(' ')
                
                if len(Subwords) == 2 and s == 0  :
                    for unit in units:
                        if unit in Subwords : s = 1

                if len(Subwords) == 1 and s == 0  :           
                    for quart in share_word :
                        if quart in word: s=1 

                if isinstance(word, str) and word.isnumeric(): s = 1
                    
                if isinstance(word, str) and word.count('.') == 1 and all(char.isdigit() or char == '.' for char in word): s = 1
                    
                if s == 0: word_filter.append(word)
                    
        return list(numpy.unique(word_filter))

    def search_share_in_table(self,words,NewsId,source):
        # connect to the database
        conn = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V2.sqlite')
        c = conn.cursor()
        share = []
        loss = []

        for word in words : 

            w = word
            if source == 'kaohoon': w = word + '.BK'

            # search for words in the table
            c.execute(f"""SELECT * FROM Information WHERE Symbol="{w}" or Sname="{word}"; """)
            results = c.fetchall()

            if results != []:
                data = {}
                data['SymbolId'],data['NewsId'] = results[0][0],NewsId
                share.append(data)

            else :

                loss.append(word)

        # close the database connection
        conn.close()

        return share,loss

    def find_location(self,words,NewsId):

        geolocator = Nominatim(user_agent="Geolocation")
        loca,extra = [],[]
        for i in words :
    
            location = geolocator.geocode(f"{i}", exactly_one=True, namedetails=True, addressdetails=True,timeout=12000, language='en')

            if str(location) != 'None' :

                data = {}

                try :
                    if i in ['Africa', 'Europe', 'Asia', 'North America', 'South America', 'Antarctica', 'Australia','South Pole','North pole'] :
                        data['Lname'],data['Country'],data['point'],data['NewsId'] = i,'',f'({location.latitude}, {location.longitude})',NewsId
                    else:
                        data['Lname'],data['Country'],data['point'],data['NewsId'] = i,location.raw['address']['country'],f'({location.latitude}, {location.longitude})',NewsId

                    loca.append(data)

                except KeyError: extra.append(i)
                    
        return loca